# UEP 239 Final Project
#### Jess Wilson - May 2021

---

### Goal of Project:
- Discover the most suitable zip code tabulation areas (ZCTAs) for young professionals to move to in Metro Boston. 

### Suitability Indicator Variables: 
1. Mean rent payment (USD)
2. Population density (population per square km)
3. Proximity to farmer's markets (km)
4. Transit stop density (stop per square km)
5. Proximity to supermarkets (km)

---

#### Instructions on How to Run:
- All instructions can be found in the README.md, including how to load and run environment and download necessary datasets 
- Environment (environment.yml) and data directory (uep239-final-project-data) are located in repository

---

#### Importing Relevant Libraries:

In [1]:
import numpy as np               # Load numpy, for scientific computing
import pandas as pd              # Load pandas, for data frame manipulation 
import geopandas as gpd          # Load geopandas, for pandas manipulation with geospatial components
import matplotlib.pyplot as plt  # Load matplotlib, for plotting and mappingdata
import seaborn as sns            # Load seaborn, for additional plotting features 
import folium                    # Load folium, for interactive maps
import os                        # Load OS, for operating system work
import contextily as cx          # Load contextily, for basemaps

---

#### Setting Working Directory:

In [6]:
# Pass the raw string (r) path of the directory in which you downloaded the project data
os.chdir(os.path.dirname(r"z:/OneDrive/Documentos/GPwP/uep239-final-project/uep239-final-project-data/"))
# Print working directory
print("Path changed to: "+os.getcwd())

Path changed to: z:\OneDrive\Documentos\GPwP\uep239-final-project\uep239-final-project-data


---

#### Read in Relevant Data Sets and Clean Data:



---

**Population:**

In [36]:
# Read in ZCTA population data:
population_raw = pd.read_csv(r"tabular/Population/population.csv", skiprows = [1])

# Observe data:
population_raw.info()
population_raw.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537 entries, 0 to 536
Columns: 458 entries, GEO_ID to S0101_C06_038M
dtypes: int64(186), object(272)
memory usage: 1.9+ MB


,GEO_ID,NAME,S0101_C01_001E,S0101_C01_001M,S0101_C01_002E,S0101_C01_002M,S0101_C01_003E,S0101_C01_003M,S0101_C01_004E,S0101_C01_004M,...,S0101_C06_034E,S0101_C06_034M,S0101_C06_035E,S0101_C06_035M,S0101_C06_036E,S0101_C06_036M,S0101_C06_037E,S0101_C06_037M,S0101_C06_038E,S0101_C06_038M
0,8600000US01001,ZCTA5 01001,17312,735,956,233,863,222,967,274,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
1,8600000US01002,ZCTA5 01002,30014,485,872,143,1043,192,1123,215,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
2,8600000US01003,ZCTA5 01003,11357,477,0,19,0,19,0,19,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
3,8600000US01005,ZCTA5 01005,5128,404,127,72,199,139,408,153,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
4,8600000US01007,ZCTA5 01007,15005,20,879,183,828,186,969,193,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)


In [39]:
# Rename population column:
population_raw.rename(columns = {'S0101_C01_001E' : 'Total_Population'}, inplace = True)

# Subset data into only the necessary columns for analysis:
population = population_raw[["GEO_ID", "NAME", "Total_Population"]]
population.head()

,GEO_ID,NAME,Total_Population
0,8600000US01001,ZCTA5 01001,17312
1,8600000US01002,ZCTA5 01002,30014
2,8600000US01003,ZCTA5 01003,11357
3,8600000US01005,ZCTA5 01005,5128
4,8600000US01007,ZCTA5 01007,15005


---

**Monthly Rent:**

In [41]:
# Read in median monthly rent data:
rent_raw = pd.read_csv(r"tabular/Rent/rent.csv", skiprows = [1])

# Observe data:
rent_raw.info()
rent_raw.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537 entries, 0 to 536
Columns: 554 entries, GEO_ID to S2503_C06_046M
dtypes: int64(266), object(288)
memory usage: 2.3+ MB


,GEO_ID,NAME,S2503_C01_001E,S2503_C01_001M,S2503_C01_002E,S2503_C01_002M,S2503_C01_003E,S2503_C01_003M,S2503_C01_004E,S2503_C01_004M,...,S2503_C06_042E,S2503_C06_042M,S2503_C06_043E,S2503_C06_043M,S2503_C06_044E,S2503_C06_044M,S2503_C06_045E,S2503_C06_045M,S2503_C06_046E,S2503_C06_046M
0,8600000US01001,ZCTA5 01001,7413,322,230,138,163,105,320,113,...,15.6,6.9,1.6,1.8,0.0,1.7,2.1,1.7,7.9,3.5
1,8600000US01002,ZCTA5 01002,9798,451,562,198,376,130,465,212,...,9.3,3.0,10.7,3.7,2.6,2.2,6.3,3.2,1.1,0.9
2,8600000US01003,ZCTA5 01003,42,42,24,38,0,19,14,20,...,0.0,49.9,0.0,49.9,0.0,49.9,63.2,53.3,0.0,49.9
3,8600000US01005,ZCTA5 01005,1944,232,51,50,27,44,34,50,...,28.1,29.3,0.0,12.1,0.0,12.1,0.0,12.1,17.4,18.0
4,8600000US01007,ZCTA5 01007,5563,196,81,65,109,107,105,90,...,14.1,7.1,13.5,11.0,0.0,3.3,0.0,3.3,4.6,5.2


In [48]:
# Rename rent column:
rent_raw.rename(columns = {"S2503_C05_024E" : "Median_Rent"}, inplace = True)
# Replace empty values:
rent_raw = rent_raw.replace('-', np.NaN)

# Subset data into only the necessary columns for analysis:
rent = rent_raw[["GEO_ID", "NAME", "Median_Rent"]]
rent.head()

,GEO_ID,NAME,Median_Rent
0,8600000US01001,ZCTA5 01001,1148
1,8600000US01002,ZCTA5 01002,1380
2,8600000US01003,ZCTA5 01003,NaN
3,8600000US01005,ZCTA5 01005,965
4,8600000US01007,ZCTA5 01007,975


---

**Grocery Stores:**

In [49]:
# Read in grocery store data:
grocery_raw = pd.read_csv(r"tabular/Supermarkets/supermarkets.csv")

# Observe data:
grocery_raw.info()
grocery_raw.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274 entries, 0 to 273
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Company Name  274 non-null    object 
 1   Address       274 non-null    object 
 2   City          274 non-null    object 
 3   State         274 non-null    object 
 4   ZIP Code      274 non-null    int64  
 5   Latitude      274 non-null    float64
 6   Longitude     274 non-null    float64
dtypes: float64(2), int64(1), object(4)
memory usage: 15.1+ KB


,Company Name,Address,City,State,ZIP Code,Latitude,Longitude
0,99 Ranch Market,475 Hancock St # 2,Quincy,MA,2171,42.270653,-71.023292
1,Big Y,500 Staples Dr,Framingham,MA,1702,42.291206,-71.489449
2,Big Y,770 Cochituate Rd,Framingham,MA,1701,42.309146,-71.380338
3,Big Y,348 E Central St,Franklin,MA,2038,42.079076,-71.380864
4,Big Y,182 Summer St # 1,Kingston,MA,2364,42.011307,-70.735435


In [50]:
# Subset data into only the necessary columns for analysis:
grocery = grocery_raw[["Company Name", "Latitude", "Longitude"]]
grocery.head()

,Company Name,Latitude,Longitude
0,99 Ranch Market,42.270653,-71.023292
1,Big Y,42.291206,-71.489449
2,Big Y,42.309146,-71.380338
3,Big Y,42.079076,-71.380864
4,Big Y,42.011307,-70.735435


---

**MPO Boundaries:**

In [51]:
# Read in MPO boundary data:
mpo_raw = gpd.read_file(r"vector/MPO_Boundaries/mpo_boundaries.shp")

# Observe data:
mpo_raw.info()
mpo_raw.head()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   OBJECTID    13 non-null     int64   
 1   MPO         13 non-null     object  
 2   created_us  0 non-null      object  
 3   created_da  13 non-null     object  
 4   last_edite  2 non-null      object  
 5   last_edi_1  13 non-null     object  
 6   GlobalID    13 non-null     object  
 7   ShapeSTAre  13 non-null     float64 
 8   ShapeSTLen  13 non-null     float64 
 9   geometry    13 non-null     geometry
dtypes: float64(2), geometry(1), int64(1), object(6)
memory usage: 1.1+ KB


,OBJECTID,MPO,created_us,created_da,last_edite,last_edi_1,GlobalID,ShapeSTAre,ShapeSTLen,geometry
0,2,Berkshire,None,1970-01-01,None,1970-01-01,{08FDA544-18B0-412A-B442-287E53E987F7},2.451015e+09,2.471530e+05,"POLYGON ((-8128884.676 5272654.345, -8128962.2..."
1,3,Cape Cod,None,1970-01-01,None,1970-01-01,{B6CD90CF-2F7D-43F2-B251-FA7F8E00EF01},1.067067e+09,1.288227e+06,"MULTIPOLYGON (((-7813968.781 5173329.197, -781..."
2,4,Central Massachusetts,None,1970-01-01,None,1970-01-01,{CC777E14-53C8-42AD-B421-71444DA0BB60},2.487546e+09,2.683265e+05,"POLYGON ((-7977225.352 5223837.273, -7973861.8..."
3,5,Franklin,None,1970-01-01,None,1970-01-01,{4804E708-6B89-4A85-9383-BD91F7589981},1.876456e+09,2.527017e+05,"POLYGON ((-8046511.241 5269691.856, -8045276.8..."
4,6,Montachusett,None,1970-01-01,None,1970-01-01,{F315DA63-C9CF-40EE-8AA7-5ABA2E1FD528},1.772355e+09,2.748684e+05,"POLYGON ((-7976246.504 5267152.001, -7976121.9..."


In [54]:
# Filter MPO to Boston Region MPO:
mpo_raw = mpo_raw[mpo_raw.MPO == "Boston Region"]

# Subset data into only the necessary columns for analysis:
mpo = mpo_raw[["OBJECTID", "MPO", "ShapeSTAre", "ShapeSTLen", "geometry"]]
mpo.head()

,OBJECTID,MPO,ShapeSTAre,ShapeSTLen,geometry
10,12,Boston Region,3.524379e+09,1.665026e+06,"MULTIPOLYGON (((-7875339.226 5247387.185, -787..."


---

**MA Town Boundaries:**

In [ ]:
# Read in MA town boundary data:
town_raw = gpd.read_file(r"vector/Town_Boundaries/town_boundaries.shp")

# Observe data:
town_raw.info()
town_raw.head()
town = 

---

**ZCTA Boundaries:**

In [ ]:
# Read in ZCTA boundary data:
zcta_raw = gpd.read_file(r"vector/ZCTA_Boundaries/zcta_boundaries.shp")

# Observe data:
zcta_raw.info()
zcta_raw.head()
zcta = 

---

**Bus Stops:**

In [ ]:
# Read in bus stop data:
bus_raw = gpd.read_file(r"vector/Bus_Stops/bus_stops.shp")

#Observe data:
bus_raw.info()
bus_raw.head()
bus = 

---

**Transit Stops:**

In [ ]:
# Read in transit stop data:
transit_raw = gpd.read_file(r"vector/Transit_Stops/transit_stops.shp")

# Observe data:
transit_raw.info()
transit_raw.head()
transit = 

---

**Farmer's Markets:**

In [ ]:
# Read in farmer's markets data:
market_raw = gpd.read_file(r"vector/Farmers_Markets/farmers_markets.shp")

# Observe data:
market_raw.info()
market_raw.head()
market = 

---

#### Data Manipulation, Joining, and Reprojecting:

In [4]:
# join MPO with zcta (intersect) and join zcta with town (keep all)
# population and rent joined to zcta 
# join transit with bus (keep all)
# convert grocery to gpd using lat/lon

---

#### Creation of Basemap:

In [ ]:
# zcta map feat. town and mpo
# use contextily (cx)

---

#### Analysis of Suitability Indicator Variables:

1. Mean rent payment (USD)
2. Population density (population per square km)
3. Proximity to farmer's markets (km)
4. Transit stop density (stop per square km)
5. Proximity to supermarkets (km)

---

**1. Mean rent payment (USD):**

In [ ]:
# Visualize spatial data - median rent per ZCTA:

In [ ]:
# Summarize indicator values - mean/median rent per ZCTA:

In [ ]:
# Visualize mean/med rent per ZCTA

In [ ]:
# Produce ZCTA ranking based on mean/med rent, report highest and lowest ranking ZCTAs (e.g. top 5 and bottom 5)

In [ ]:
# Convert indicator values into suitability score by normalizing values into suitability index ranging from zero to one

---

**2. Population density (population per square km):**

In [ ]:
# Visualize spatial data - population per ZCTA

In [ ]:
# Summarize indicator values - population per square km per ZCTA:
# Density per ZCTA function

In [ ]:
# Visualize pop density per ZCTA

In [ ]:
# Produce ZCTA ranking based on mean/med rent, report highest and lowest ranking ZCTAs (e.g. top 5 and bottom 5)

In [ ]:
# Convert indicator values into suitability score by normalizing values into suitability index ranging from zero to one

---

**3. Proximity to farmer's markets (km):**

In [ ]:
# Visualize spatial data - farmer's market locations:

In [ ]:
# Summarize indicator values - closest farmer's market per ZCTA (euc dis):
# Closest POI function

In [ ]:
# Visualize closest farmer's market per ZCTA

In [ ]:
# Produce ZCTA ranking based on mean/med rent, report highest and lowest ranking ZCTAs (e.g. top 5 and bottom 5)

In [ ]:
# Convert indicator values into suitability score by normalizing values into suitability index ranging from zero to one

---

**4. Transit stop density (stop per square km):**

In [ ]:
# Visualize spatial data - bus and transit stop locations:

In [ ]:
# Summarize indicator values - stop per square km per ZCTA:
# Density per ZCTA function

In [ ]:
# Visualize transit stop density per ZCTA

In [ ]:
# Produce ZCTA ranking based on mean/med rent, report highest and lowest ranking ZCTAs (e.g. top 5 and bottom 5)

In [ ]:
# Convert indicator values into suitability score by normalizing values into suitability index ranging from zero to one

---

**5. Proximity to supermarkets (km):**

In [ ]:
# Visualize spatial data - grocery store locations:

In [ ]:
# Summarize indicator values - closest supermarket per ZCTA (euc dis):
# Closest POI function

In [ ]:
# Visualize closest supermarket per ZCTA

In [ ]:
# Produce ZCTA ranking based on mean/med rent, report highest and lowest ranking ZCTAs (e.g. top 5 and bottom 5)

In [ ]:
# Convert indicator values into suitability score by normalizing values into suitability index ranging from zero to one

---

#### Unweighted Suitability Index:

- Mean rent payment (USD) = 25%
- Population density (population per square km) = 25%
- Proximity to farmer's markets (km) = 25%
- Transit stop density (stop per square km) = 25%
- Proximity to supermarkets (km) = 25%

In [ ]:
unweighted = rent[score] + population[score] + market[score] + transit[score] + grocery [score]
unweighted_index_top = unweighted.sort_values('score',ascending=False).head()
unweighted_index_bottom = unweighted.sort_values('score',ascending=False).tail()

In [ ]:
#plot unweighted
#Interactive map needs to be in WGS84 CRS

---

#### Weighted Suitability Index:

- Mean rent payment (USD) = 47.5%
- Population density (population per square km) = 6.5%
- Proximity to farmer's markets (km) = 3.3%
- Transit stop density (stop per square km) = 13.5%
- Proximity to supermarkets (km) = 29.2%

*Weights attributed using AHP method: https://bpmsg.com/ahp/ahp-calc.php*


In [ ]:
weighted = rent[score] * 0.475 + population[score] * 0.065 + market[score] * 0.033 + transit[score] * 0.135 + grocery [score] * 0.292
weighted_index_top = weighted.sort_values('score',ascending=False).head()
weighted_index_bottom = weighted.sort_values('score',ascending=False).tail()

In [ ]:
#plot weighted
#Interactive map needs to be in WGS84 CRS